In [13]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from pypfopt import BlackLittermanModel

# Read in price data
df = pd.read_csv("stock_prices.csv", parse_dates=True, index_col="date")

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S) #Add constraints: ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.1)) #Max 10% on each weight.
ef.add_objective(objective_functions.L2_reg, gamma=1) #Reduce zero-weights by adding penalty on small weights with a L2-regularization term
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict({'GOOG': 0.08369, 'AAPL': 0.09537, 'FB': 0.11076, 'BABA': 0.067, 'AMZN': 0.10739, 'GE': 0.02716, 'AMD': 0.0, 'WMT': 0.03077, 'BAC': 0.0, 'GM': 0.0, 'T': 0.02196, 'UAA': 0.01522, 'SHLD': 0.0, 'XOM': 0.0432, 'RRC': 0.0, 'BBY': 0.06553, 'MA': 0.1515, 'PFE': 0.08498, 'JPM': 0.0251, 'SBUX': 0.07038})
Expected annual return: 26.5%
Annual volatility: 21.7%
Sharpe Ratio: 1.13


/home/codespace/.python/current/lib/python3.12/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


(0.2650387590869301, 0.21662532560561654, 1.1311639504839912)

In [11]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices


latest_prices = get_latest_prices(df)

da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'MA': 20, 'FB': 12, 'PFE': 54, 'BABA': 4, 'AAPL': 4, 'GOOG': 1, 'BBY': 2, 'SBUX': 1}
Funds remaining: $11.89


### Black-Litterman allocation

In [14]:
S = risk_models.sample_cov(df)
viewdict = {"AAPL": 0.20, "BBY": -0.30, "BAC": 0, "SBUX": -0.2, "T": 0.131321}
bl = BlackLittermanModel(S, pi="equal", absolute_views=viewdict, omega="default")
rets = bl.bl_returns()

ef = EfficientFrontier(rets, S)
ef.max_sharpe()

OrderedDict([('GOOG', 0.0),
             ('AAPL', 0.2516700243034513),
             ('FB', 0.0061410740896274),
             ('BABA', 0.0504178900290811),
             ('AMZN', 0.0),
             ('GE', 0.0),
             ('AMD', 0.0),
             ('WMT', 0.0),
             ('BAC', 0.0),
             ('GM', 0.0),
             ('T', 0.6703436046652544),
             ('UAA', 0.0),
             ('SHLD', 0.0),
             ('XOM', 0.0214274069125859),
             ('RRC', 0.0),
             ('BBY', 0.0),
             ('MA', 0.0),
             ('PFE', 0.0),
             ('JPM', 0.0),
             ('SBUX', 0.0)])

In [ ]:
from forex_python.converter import CurrencyRates
import pandas as pd

# Initialize the CurrencyRates object
c = CurrencyRates()

# Example dataframe with dates and prices in EUR
df = pd.DataFrame({
    'date': ['2023-01-01', '2023-01-02'],
    'price_eur': [100, 105]
})

# Convert prices to USD
df['price_usd'] = df.apply(lambda row: row['price_eur'] * c.get_rate('EUR', 'USD', row['date']), axis=1)
print(df)



- Frontline (FRO.OL): NOK (listed on the Oslo Stock Exchange)
- Nordea Bank Abp (NDA-FI.HE): EUR (listed on the Helsinki Stock Exchange)
- BAWAG Group AG (BG.VI): EUR (listed on the Vienna Stock Exchange)
- CaixaBank (CABK.MC): EUR (listed on the Madrid Stock Exchange)
- Danske Bank (DANSKE.CO): DKK (listed on the Copenhagen Stock Exchange)
- McDonald’s (MCD): USD (listed on the NYSE)



In [47]:
import yfinance as yf
import pandas as pd

# List of ticker symbols
tickers = ['FRO.OL', 'NDA-FI.HE', 'BG.VI', 'CABK.MC', 'DANSKE.CO', 'MCD']

# Download historical data for the specified tickers
data = yf.download(tickers, start='2008-01-01', end='2024-09-10')

# Display the first few rows of the data
display(data.head())



[*********************100%***********************]  6 of 6 completed


Price                     Adj Close                                     \
Ticker                        BG.VI   CABK.MC   DANSKE.CO       FRO.OL   
Date                                                                     
2008-01-02 00:00:00+00:00       NaN  2.082304  122.023254  1174.019409   
2008-01-03 00:00:00+00:00       NaN  2.042260  120.215508  1180.703003   
2008-01-04 00:00:00+00:00       NaN  2.042260  117.955788  1122.782104   
2008-01-07 00:00:00+00:00       NaN  1.982193  119.010353  1060.405029   
2008-01-08 00:00:00+00:00       NaN  2.002215  120.215508  1075.998901   

Price                                          Close                      \
Ticker                           MCD NDA-FI.HE BG.VI CABK.MC   DANSKE.CO   
Date                                                                       
2008-01-02 00:00:00+00:00  36.280487  4.380477   NaN    5.20  187.387222   
2008-01-03 00:00:00+00:00  36.174339  4.392221   NaN    5.10  184.611115   
2008-01-04 00:00:00+00:00  35.624817  4.306099   NaN    5.10  181.140976   
2008-01-07 00:00:00+00:00  36.236778  4.321758   NaN    4.95  182.760376   
2008-01-08 00:00:00+00:00  35.643559  4.321758   NaN    5.00  184.611115   

Price                              ...        Open                     \
Ticker                     FRO.OL  ...   DANSKE.CO  FRO.OL        MCD   
Date                               ...                                  
2008-01-02 00:00:00+00:00  1317.5  ...  184.842453  1302.5  59.480000   
2008-01-03 00:00:00+00:00  1325.0  ...  186.461853  1315.0  58.400002   
2008-01-04 00:00:00+00:00  1260.0  ...  184.611115  1317.5  57.330002   
2008-01-07 00:00:00+00:00  1190.0  ...  180.909637  1247.5  57.360001   
2008-01-08 00:00:00+00:00  1207.5  ...  184.148422  1160.0  58.439999   

Price                               Volume                                  \
Ticker                    NDA-FI.HE  BG.VI    CABK.MC  DANSKE.CO    FRO.OL   
Date                                                                         
2008-01-02 00:00:00+00:00     11.36    NaN  3370267.0   239259.0  131921.0   
2008-01-03 00:00:00+00:00     11.18    NaN  2046484.0   427514.0  145772.0   
2008-01-04 00:00:00+00:00     11.20    NaN  3184218.0   396353.0  244393.0   
2008-01-07 00:00:00+00:00     11.00    NaN  4010529.0   446996.0  248477.0   
2008-01-08 00:00:00+00:00     11.11    NaN  7756402.0  2476565.0  260464.0   

Price                                             
Ticker                            MCD  NDA-FI.HE  
Date                                              
2008-01-02 00:00:00+00:00   7858300.0   550381.0  
2008-01-03 00:00:00+00:00   6514700.0   748733.0  
2008-01-04 00:00:00+00:00   9687500.0  2332216.0  
2008-01-07 00:00:00+00:00  10784500.0  2407069.0  
2008-01-08 00:00:00+00:00  10080800.0   451652.0  

[5 rows x 36 columns]

In [57]:
data_reset=data.stack(level=0).reset_index()



/tmp/ipykernel_1691/3137080973.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data_reset=data.stack(level=0).reset_index()


In [59]:
display(data_reset.query('Price == "Adj Close"'))

Ticker,Date,Price,BG.VI,CABK.MC,DANSKE.CO,FRO.OL,MCD,NDA-FI.HE
0,2008-01-02 00:00:00+00:00,Adj Close,NaN,2.082304,122.023254,1174.019409,36.280487,4.380477
6,2008-01-03 00:00:00+00:00,Adj Close,NaN,2.042260,120.215508,1180.703003,36.174339,4.392221
12,2008-01-04 00:00:00+00:00,Adj Close,NaN,2.042260,117.955788,1122.782104,35.624817,4.306099
18,2008-01-07 00:00:00+00:00,Adj Close,NaN,1.982193,119.010353,1060.405029,36.236778,4.321758
24,2008-01-08 00:00:00+00:00,Adj Close,NaN,2.002215,120.215508,1075.998901,35.643559,4.321758
...,...,...,...,...,...,...,...,...
25836,2024-09-03 00:00:00+00:00,Adj Close,67.599998,5.310000,208.399994,243.567825,285.519989,10.615000
25842,2024-09-04 00:00:00+00:00,Adj Close,68.500000,5.288000,210.399994,241.273773,288.029999,10.515000
25848,2024-09-05 00:00:00+00:00,Adj Close,70.900002,5.374000,209.699997,238.082062,287.970001,10.685000
25854,2024-09-06 00:00:00+00:00,Adj Close,69.099998,5.318000,207.699997,241.572998,289.510010,10.435000


In [68]:
data_reset_index=data_reset.reset_index(drop = True)
display(data_reset_index[['Date', 'Price']].head())
print(data_reset_index.columns)

Ticker,Date,Price
0,2008-01-02 00:00:00+00:00,Adj Close
1,2008-01-02 00:00:00+00:00,Close
2,2008-01-02 00:00:00+00:00,High
3,2008-01-02 00:00:00+00:00,Low
4,2008-01-02 00:00:00+00:00,Open


Index(['Date', 'Price', 'BG.VI', 'CABK.MC', 'DANSKE.CO', 'FRO.OL', 'MCD',
       'NDA-FI.HE'],
      dtype='object', name='Ticker')


In [56]:
display(data_reset[['Date', 'Price', 'BG.VI', 'CABK.MC', 'DANSKE.CO', 'FRO.OL', 'MCD', 'NDA-FI.HE']].query('Price == "Adj Close"').head())
newdata=data_reset[['Date', 'Price', 'BG.VI', 'CABK.MC', 'DANSKE.CO', 'FRO.OL', 'MCD', 'NDA-FI.HE']].query('Price == "Adj Close"')
newdata_reset=newdata.set_index('Date', drop = True)
display(newdata_reset[['Date', 'Price', 'BG.VI']])

Ticker,Date,Price,BG.VI,CABK.MC,DANSKE.CO,FRO.OL,MCD,NDA-FI.HE
0,2008-01-02 00:00:00+00:00,Adj Close,NaN,2.082304,122.023254,1174.019409,36.280487,4.380477
6,2008-01-03 00:00:00+00:00,Adj Close,NaN,2.042260,120.215508,1180.703003,36.174339,4.392221
12,2008-01-04 00:00:00+00:00,Adj Close,NaN,2.042260,117.955788,1122.782104,35.624817,4.306099
18,2008-01-07 00:00:00+00:00,Adj Close,NaN,1.982193,119.010353,1060.405029,36.236778,4.321758
24,2008-01-08 00:00:00+00:00,Adj Close,NaN,2.002215,120.215508,1075.998901,35.643559,4.321758


KeyError: "['Date'] not in index"

In [46]:
newdata_reset.set_index('Date', inplace=True)
display(newdata_reset)

Ticker,index,Price,BG.VI,CABK.MC,DANSKE.CO,FRO.OL,MCD,NDA-FI.HE
Date,,,,,,,,
2008-01-02 00:00:00+00:00,0,Adj Close,NaN,2.082304,122.023254,1174.019409,36.280487,4.380477
2008-01-03 00:00:00+00:00,6,Adj Close,NaN,2.042260,120.215508,1180.703003,36.174339,4.392221
2008-01-04 00:00:00+00:00,12,Adj Close,NaN,2.042260,117.955788,1122.782104,35.624817,4.306099
2008-01-07 00:00:00+00:00,18,Adj Close,NaN,1.982193,119.010353,1060.405029,36.236778,4.321758
2008-01-08 00:00:00+00:00,24,Adj Close,NaN,2.002215,120.215508,1075.998901,35.643559,4.321758
...,...,...,...,...,...,...,...,...
2024-09-03 00:00:00+00:00,25836,Adj Close,67.599998,5.310000,208.399994,243.567825,285.519989,10.615000
2024-09-04 00:00:00+00:00,25842,Adj Close,68.500000,5.288000,210.399994,241.273773,288.029999,10.515000
2024-09-05 00:00:00+00:00,25848,Adj Close,70.900002,5.374000,209.699997,238.082062,287.970001,10.685000


In [2]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 59.7 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-py3-none-any.whl size=138891 sha256=7268ab58d7e33080c6514d98a0a74081eedae6ec00d12d55f1d521a4e699374c
  Stored in directory: /home/codespace/.cache/pip/wheels/a6/5e/0f/8319805c4115320e0d3e8fb5799b114a2e4c4a3d6c7e523b06
Successfully built peewee


In [70]:
# Sample dataframe with Ticker as index
data = {
    'Date': ['2021-01-01', '2021-01-02', '2021-01-03'],
    'Price': [150, 250, 2800],
    'BG.VI': [100, 200, 300],
    'CABK.MC': [110, 210, 310],
    'DANSKE.CO': [120, 220, 320],
    'FRO.OL': [130, 230, 330],
    'MCD': [140, 240, 340],
    'NDA-FI.HE': [150, 250, 350]
}
df = pd.DataFrame(data)

# Set 'Date' as the index
df.set_index('Date', inplace=True)

print(df.columns)


Index(['Price', 'BG.VI', 'CABK.MC', 'DANSKE.CO', 'FRO.OL', 'MCD', 'NDA-FI.HE'], dtype='object')
